In [ ]:
# %%
# filename: strat_supertrend_initial_backtest.ipynb

# --- 1. Imports and Setup ---
import os
import pandas as pd
import pandas_ta as ta
import vectorbt as vbt
import plotly.graph_objects as go

In [ ]:
# %%
# --- 2. Configuration and Data Loading ---

# --- Path Configuration ---
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
data_dir = os.path.join(project_root, "data", "parquet")
# Make sure this filename matches the file you copied into the data/parquet directory
DATA_FILENAME = "EURUSD_M15_2024-09-14_to_2025-09-14.parquet"
file_path = os.path.join(data_dir, DATA_FILENAME)

# --- Strategy Parameters ---
SUPERTREND_PERIOD = 7
SUPERTREND_MULTIPLIER = 3.0

# --- Data Loading ---
print(f"Loading data from: {file_path}")
try:
    price_data = pd.read_parquet(file_path)
    price_data.set_index('Time', inplace=True)
    print("Data loaded successfully and index set.")
except FileNotFoundError:
    print(f"ERROR: File not found at '{file_path}'.")
    # Stop execution if data is not found
    exit()

In [ ]:
# %%
# --- 3. Indicator Calculation ---
print(f"Calculating Supertrend with period={SUPERTREND_PERIOD} and multiplier={SUPERTREND_MULTIPLIER}...")

# Use pandas-ta to calculate the Supertrend indicator
# It conveniently returns multiple columns, including the trend direction and the line itself.
price_data.ta.supertrend(
    length=SUPERTREND_PERIOD,
    multiplier=SUPERTREND_MULTIPLIER,
    append=True
)

# For clarity, let's rename the supertrend line column
# The default name is SUPERT_7_3.0
supertrend_col_name = f"SUPERT_{SUPERTREND_PERIOD}_{SUPERTREND_MULTIPLIER}"
price_data.rename(columns={supertrend_col_name: 'supertrend_line'}, inplace=True)

print("Supertrend calculation complete.")
print(price_data.tail())

In [ ]:
# %%
# --- 4. Strategy Logic & Signal Generation ---
print("Generating trading signals...")

# The signal is when the Close price crosses the Supertrend line.
# We use vectorbt's built-in signal functions for this.
entries = price_data['Close'].vbt.crossed_above(price_data['supertrend_line'])
exits = price_data['Close'].vbt.crossed_below(price_data['supertrend_line'])

In [ ]:
# %%
# --- 5. Backtesting and Analysis ---
print("Running backtest with vectorbt...")

# Create the portfolio from the generated signals
portfolio = vbt.Portfolio.from_signals(
    price_data['Close'],
    entries,
    exits,
    freq='15min', # Set the frequency of the data (15 minutes)
    init_cash=10000,
    fees=0.0001,    # Example fee: 0.01%
    slippage=0.0001 # Example slippage: 0.01%
)

# Print the performance statistics
print("\n--- Backtest Results ---")
print(portfolio.stats())

In [ ]:
# %%
# --- 6. Visualization (Optimized and Corrected) ---
print("\nGenerating plot...")

# --- Performance Optimization for Plotting ---
# To avoid browser slowdowns, we will only plot the last 3 months of data.
# We use the modern `.loc` slicing method instead of the deprecated `.last()`.
print("Selecting the last 3 months of data for visualization...")
end_date = price_data.index[-1]
start_date_plot = end_date - pd.DateOffset(months=3)
plot_data = price_data.loc[start_date_plot:]

# --- Re-run Backtest on the Sliced Data for Clean Plotting ---
# This is the most robust way to get a portfolio object that matches the plot data.
# It's extremely fast, so there's no significant performance cost.
print("Running a quick backtest on the sliced data for plotting...")
plot_entries = entries.loc[plot_data.index]
plot_exits = exits.loc[plot_data.index]

plot_portfolio = vbt.Portfolio.from_signals(
    plot_data['Close'],
    plot_entries,
    plot_exits,
    freq='15min', # Use the corrected frequency string
    init_cash=10000,
    fees=0.0001,
    slippage=0.0001
)

# --- Layered Plotting ---
# Step 1: Create the base candlestick chart from the SLICED data
fig = plot_data.vbt.ohlc.plot(
    title_text=f'Supertrend Strategy on {DATA_FILENAME} (Last 3 Months)',
    template='plotly_dark'
)

# Step 2: Add the Supertrend line from the SLICED data
fig.add_trace(
    go.Scatter(
        x=plot_data.index,
        y=plot_data['supertrend_line'],
        mode='lines',
        name='Supertrend Line',
        line=dict(color='yellow', width=1.5)
    )
)

# Step 3: Add the trade markers from the NEWLY created plot_portfolio
plot_portfolio.trades.plot(fig=fig)

# Step 4: Show the final, combined figure
fig.show()